# Examples of anonymized datasets

<a target="_blank" href="https://colab.research.google.com/github/apiwat-chantawibul/anon-workshop/blob/main/notebooks/examples/examples.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

- ขอยกตัวอย่างข้อมูลตั้งต้นชุดเดียว
- แต่แสดงผลลัพธ์ของการทำ anonymization X รูปแบบหลักๆ คือ
  - Publish ในรูปแบบตาราง
  - Publish ในรูปแบบสถิติ
  - เปิด API ให้เรียกถามเป็นครั้งๆไป
- เพื่อให้มีภาพเป้าหมายอยู่ในใจ และเห็นความเป็นไปได้หลายๆแบบก่อน
- จากนั้นเราค่อยมาเรียนรู้ข้อดี, ข้อเสีย, และวิธีการทำ anonymization แต่ละรูปแบบ

## ข้อมูลตั้งต้น

- เป็นข้อมูลจำลอง
- ถูกสร้างมาจาก [generate_mock_source.ipynb](generate_mock_source.ipynb)
- แล้วเก็บไว้เป็น file [source.csv](source.csv)
  - ⚠️ ถ้ารันบน Colab ต้อง download file ก่อน

In [2]:
!curl -OLf https://github.com/apiwat-chantawibul/anon-workshop/raw/refs/heads/main/notebooks/examples/source.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  571k  100  571k    0     0   498k      0  0:00:01  0:00:01 --:--:-- 22.7M


### ลักษณะข้อมูลตั้งต้น

- สมมุติว่าเป็นข้อมูลจำนวนเงินที่สมาชิกในโครงการหนึ่งๆเบิกใช้จ่ายไป และความเห็นของสมาชิกประกอบการใช้จ่ายนั้นๆ
- มี attribute ที่ระบุถึงตัวคนเจ้าของข้อมูลโดยตรง (direct identfier) อยู่สามอย่าง คือ
  - passport
  - first_name
  - last_name
- มี atttribute ที่ระบุตัวตนได้ทางอ้อม (indirect identifier) เพราะมีความเป็นไปได้สูงว่าจะประกฎในชุดข้อมูลอื่น คือ
  - date_of_birth
  - postcode
  - gender
- มี attribute อื่นๆที่ไม่น่าจะพบได้ในแหล่งข้อมูลอื่น เนื่องจากเป็นข้อมูลเฉพาะกับ คือ
  - spending --- คือ จำนวนเงินที่ได้ใช้จ่ายไป
  - comment --- คือ ความเห็นของสมาชิกประกอบการใช้จ่าย

In [3]:
import pandas as pd

original = pd.read_csv(
    'source.csv',
    parse_dates = ['date_of_birth'],
    dtype = {'postcode': str},
)
original

,passport,first_name,last_name,job,date_of_birth,comment,postcode,gender,spending
0,694369456,ปัตถพงษ์,ถนัดรบ,ศิลปิน,1963-03-09,กำไรซ่อนโน่นห่วงใยและทั่วชาว ฤดูคนตายหมดคู่ทุ่...,75320,Female,10541
1,229512554,เมษา,ทันยุค,เจ้าหน้าที่รัฐบาล,1998-01-30,น้ำเย็นกำเทเลแกรมฝาก ตะปูตีนอุปโภคโดยหยอกครูขี...,21410,Male,13130
2,U96857223,พัชรพร,ตระกูลบุญ,ผู้จัดพิมพ์,2021-12-31,ดูฝากร่มรื่นถางหน้ากาก คำสั่งรากแต่แม่พยาธิ \n...,75320,Male,11953
3,H26958060,ประจิน,ทวีเดช,นักสังคมศาสตร์,1943-01-20,สีแข็งอัศจรรย์มิตรรสชาติดึกหมัด \nดีแพ้คำถามสถ...,95840,Female,12285
4,144271331,กิติวัฒน์,บุญญาไลย์,ศิลปิน,1960-11-25,กิ่งไม้ไข่พับกว้าง ยาเขยื้อนสำหรับแกสดชื่นโรงส...,95840,Male,10399
...,...,...,...,...,...,...,...,...,...
995,D76609557,โกมล,ดำริห์ชอบ,นักการศึกษา,2020-06-05,เย็นขนมตรวจเลี้ยวองค์น้าแรก เศษอินสตาแกรมกู ทำ...,95840,Female,8251
996,R93321926,นิมุ,นักสำหรวจ,บรรณารักษ์,1913-01-31,พิพักพิพ่วนสหภาพนักเรียนความกิ่งไม้ ขุดแรกโก๋แ...,88530,Male,12172
997,134186584,สุพัตร,เนื้อนุ่ม,นักสังคมสงเคราะห์,2021-10-26,วิ่งสุดท้ายหนุนข้างแพ้รอปู คบมะเขือเทศพับถางลู...,57110,Female,9902
998,T25822912,จินต์จุฑา,ธรรมนิยม,นักสังคมสงเคราะห์,1973-10-12,บันไดขันน้ำผ่อวิชาชีพผลิตขี้ดินขาดเคิ่งย่อ ตอก...,57110,Female,10554


## Publish ในรูปแบบตาราง

- จะเห็นว่าข้อมูลถูกดังกล่าวถูกปรับหลายประการ ทำให้ไม่สามารถแยกแยะเจ้าของ comment ได้ชัดเจน
  - ตัวอย่างเช่น ถึงแม้จะมีผู้ได้รายชื่อของ "นักสังคมสงเคราะห์หญิงอายุ 40 ปีที่เข้าร่วมโครงนี้" จากแหล่งอื่นมา ก็จะยังไม่สามารถฝันธงได้ว่าใครเป็นคน comment เรื่อง "ประชาธิปไตยอาหาร" กันแน่
  - plausible deniability is useful
  - แต่ก็ต้องแลกมาด้วยความแม่นยำข้อมูลที่ลดลง
- เทคนิค anonymization ที่ใช้ ได้แก่
  - de-identification
  - บอกกลุ่มอายุแทนวันเกิดตรงๆ (bucketing)
  - mask เลข postcode บางส่วน (masking)
  - ปัดเศษตัวเลขการใช้จ่ายให้เป็นจำนวนเต็มพันที่ใกล้ที่สุด (rounding)
  - ตัดรายการที่มีข้อมูลโดดเด่นไม่เข้าพวกออก (record suppression) จาก 1000 เหลือเพียง 125 รายการที่เปิดเผยได้

In [4]:
# remove direct identifiers, replacing it with 
table = original.drop(columns = ['passport', 'first_name', 'last_name'])

# convert to age group
from datetime import datetime
import numpy as np
table['age'] = datetime.now().year - table['date_of_birth'].dt.year
table['age'] = pd.cut(
    table['age'],
    bins = [0, 10, 20, 30, 40, 50, 60, np.inf],
    right = False,
)
table = table.drop(columns = 'date_of_birth')

# Round spending to nearest 1000
table['spending'] = table['spending'].round(-3)

# Mask lower postcode digits
table['postcode'] = table['postcode'].str[:-3] + 'xxx'

# Sort into groups by indirect identifiers
table = table.set_index(['gender', 'age', 'postcode', 'job']).sort_index()

# NOTE: See Proportion of records in each group first should aid decision:
#     table.index.value_counts().plot.hist()

# Suppress records belonging to small groups
table = table[(table.index.value_counts() >= 3)]

with pd.option_context('display.max_rows', None):
    display(table)

/tmp/ipykernel_79/3250330566.py:28: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  table = table[(table.index.value_counts() >= 3)]
/tmp/ipykernel_79/3250330566.py:28: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  table = table[(table.index.value_counts() >= 3)]


comment  \
gender age          postcode job                                                                       
Female [40.0, 50.0) 75xxx    นักสังคมสงเคราะห์     ย้ำรอทอดมันประสบการณ์อิฐเรียงของหวาน หล่นโรงสี...   
                             นักสังคมสงเคราะห์     ประชาธิปไตยอาหารชะนีนักเรียนธาตุแคะ ทวิตเตอร์พ...   
                             นักสังคมสงเคราะห์     เป็นตู้หึงสา ถีบถึงเคเอฟซีหมอกเนื่องจากแม้ \nค...   
       [60.0, inf)  21xxx    นักประวัติศาสตร์      ตั้งแต่ย่อกิจกรรมวิชาชีพบูชา สำนักงานแคะการเผื...   
                             นักประวัติศาสตร์      ห่วงใยตรวจคนตายริม ร้อนแท็กซี่เอสซีบีบันไดแหย่...   
                             นักประวัติศาสตร์      หาม้ายเอิดทุกข์ ชุดนอนที่ดินหรือหม้อเชื่อมิตร ...   
                    22xxx    นักสังคมสงเคราะห์     ควรหาม้ายริมประชาธิปไตย แกเอสซีบีผลัด ยึดแมลงก...   
                             นักสังคมสงเคราะห์     อายุผลิตประชาธิปไตยคนตาย หรือเป็นแม้อ่อนหวานเช...   
                             นักสังคมสงเคราะห์     ดังนี้ระยำสหภาพแตะผิดทุ่มบริโภค ศิษย์โรงสีซอยบ...   
                             นักแสดง               รถทัวร์ขันน้ำปิ่นโตดู ขัดลูกดาวเทียมนั่ง ตกลงต...   
                             นักแสดง               ตัวเมียซ่อมเบิ่งเพลงบัดนี้ข้าวเจ้าบัดนี้ หน้าท...   
                             นักแสดง               กระถางแกะไชโยกิจกรรมสาธารณรัฐเล่ม โน่นหรือหรือ...   
                             ผู้กำกับภาพยนตร์      ใจสลับแม้อินเทอร์เน็ต ไค้หันขันน้ำตะปูสม่ำเสมอ...   
                             ผู้กำกับภาพยนตร์      ปี่ซี่มิตรบาดชุดนอนอุปโภคธาตุ อนุบาลมิตรหนุนฤด...   
                             ผู้กำกับภาพยนตร์      ขัดขี่กลาง ชาวนาอธิษฐานศิษย์ดาวเทียม ทหารรัฐบา...   
                             ผู้จัดพิมพ์           กลางคืนคู่เศรษฐกิจรากสัญญาคบสีเทา วัฒนธรรมที่ด...   
                             ผู้จัดพิมพ์           ย่อทหารเสรีนิยม อิฐจะไปพั่งได้แก่จากผลัดขัด \n...   
                             ผู้จัดพิมพ์           วาดเจ็บไข้สัญญา ข้าวในเกมทุ่มหน้าต่างยา เฟซบุ๊...   
                    57xxx    คนขับรถแท็กซี่        หน้าต่างแก้วอ่อนหวานดี ชุดนอนองค์น้ำเย็นวิชาชี...   
                             คนขับรถแท็กซี่        เพดานหม้อคอกอธิษฐานสุดท้ายเปลืองกรรมการแพ้ อิฐ...   
                             คนขับรถแท็กซี่        แรงงานกลางคืนชนิดแต่ง กว้างชาวประพฤติไค้หันบูช...   
                    66xxx    ภัณฑารักษ์            ตลอดหนุนสัญญากบ เกาหลีร่มรื่นผ่อศิษย์หลงองค์ \...   
                             ภัณฑารักษ์            อวัยวะอาสาหน้าต่างอินสตาแกรมแจ่มใสทุกข์ กรรมกา...   
                             ภัณฑารักษ์            เย็บเงาเมล็ดมอบเชื่อมแจ่มใสเคเอฟซี ตีนอิฐไหนเพ...   
                    75xxx    นักการทูต             หุ้มชะนีมิตรแก่ห่มผ้าตำลึง โก๋แก่เข้ารหัสกับแท...   
                             นักการทูต             เอสซีบีอายุแต่งแอนดรอยด์แตะเป้า เย็บอินสตาแกรม...   
                             นักการทูต             คบวัฒนธรรมดีทุ่มขนมตับเยอะแหย่ รู้จักอวัยวะโผล...   
                             นักสังคมศาสตร์        กระจายเขยื้อนหาย กระจายสมององค์นั่งสดชื่นกลสาธ...   
                             นักสังคมศาสตร์        ชะนีจากจ๋าไค้หันคำถามชุมนุม ผิดวิ่งเกาหลีคู่ผล...   
                             นักสังคมศาสตร์        เมตตาด้ายบูด แหย่สาธารณรัฐฉีดผลชูยูทูบ วิกิพีเ...   
                             นักสัตววิทยา          นั่งกระถางโดยอายุเป้าปกครองง่วงจัด \nสถานีเป้า...   
                             นักสัตววิทยา          แกะหุ้มทอดมันต่อไต้หวันสารวัตร กามกีฬาเศษอาหาร...   
                             นักสัตววิทยา          ผลัดคอกเพลงข้าวสีเทา ปูตอกเจ็บไข้เพดาน \nกล่าว...   
                             นักเขียน              วิ่งซี่ใจกิจกรรมเป้าหาม้ายมะเขือเทศอาสา เกาหลี...   
                             นักเขียน              เย็บหายกูบอก กระถางเข้ารหัสกูผักขันน้ำห่มผ้าสห...   
                             นักเขียน              งมศาลากลางสีกะละมังหลังคา เศษกูต้น ถีบอบอุ่นกู...   
                             นักแสดง               พ่อตอกประชาธิปไตย คู่อาหารโดยเศษอาหารหรือ แอนด...   
                             นักแสดง 